In [ ]:
!pip install transformers datasets peft trl bitsandbytes accelerate


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.4/491.4 kB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 348.0/348.0 kB 26.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 71.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 71.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 55.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments
from peft import get_peft_model, LoraConfig, TaskType
from datasets import load_dataset
from trl import SFTTrainer, DataCollatorForCompletionOnlyLM

## Data Preprocessing

In [ ]:
import pandas as pd
import json

# Load CSV (adjust encoding if needed)
csv_path = "taylor_swift_lyrics.csv"
df = pd.read_csv(csv_path, encoding="ISO-8859-1")

# Check required columns exist
assert 'track_title' in df.columns and 'lyric' in df.columns, "Missing expected columns in CSV."

# Group lyrics by track
grouped = df.groupby('track_title')['lyric'].apply(list)

# Build instruction-format JSON objects
output_data = []
for title, lyrics in grouped.items():
    for i in range(0, len(lyrics), 3):  # group every 3 lines
        lines = lyrics[i:i+3]
        if len(lines) < 2:
            continue  # skip too-short chunks
        example = {
            "instruction": "Write a Taylor Swift-style lyric:",
            "input": "",
            "output": "\n".join(line.strip() for line in lines if isinstance(line, str))
        }
        output_data.append(example)

# Save to JSONL
jsonl_path = "taylor_swift_lyrics_finetune.jsonl"
with open(jsonl_path, "w", encoding="utf-8") as f:
    for item in output_data:
        f.write(json.dumps(item, ensure_ascii=False) + "\n")

print(f"✅ Saved {len(output_data)} examples to: {jsonl_path}")

✅ Saved 1622 examples to: taylor_swift_lyrics_finetune.jsonl


In [ ]:
# Load base TinyLLaMA model
model_name = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    load_in_4bit=True,
    device_map="auto",
    trust_remote_code=True
)

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


In [ ]:
# LoRA configuration
lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    lora_dropout=0.1,
    bias="none",
    task_type=TaskType.CAUSAL_LM
)
model = get_peft_model(model, lora_config)


In [ ]:
# Load fine-tuning dataset
dataset = load_dataset("json", data_files="taylor_swift_lyrics_finetune.jsonl")

# Training args
training_args = TrainingArguments(
    output_dir="./tinyllama-lyrics-lora",
    per_device_train_batch_size=4,
    num_train_epochs=3,
    fp16=True,
    logging_steps=10,
    save_strategy="epoch",
    save_total_limit=2
)



In [ ]:
def format_example(example):
    prompt = f"### Instruction:\n{example['instruction']}\n### Response:\n{example['output']}"
    return {"text": prompt}

dataset = dataset.map(format_example)

Map:   0%|          | 0/1622 [00:00<?, ? examples/s]

In [ ]:
# Optional: remove prompt tokens from loss
data_collator = DataCollatorForCompletionOnlyLM(
    tokenizer=tokenizer,
    instruction_template="### Instruction:\n{instruction}\n### Response:\n",
    response_template="{output}") # Add response_template)

# Trainer
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset["train"],
    #tokenizer=tokenizer,
    data_collator=data_collator,
    args=training_args,
    #dataset_text_field="output"
)

trainer.train()

/usr/local/lib/python3.11/dist-packages/trl/trainer/utils.py:123: UserWarning: The pad_token_id and eos_token_id values of this tokenizer are identical. If you are planning for multi-turn training, it can result in the model continuously generating questions and answers without eos token. To avoid this, set the pad_token_id to a different value.
  warnings.warn(


Converting train dataset to ChatML:   0%|          | 0/1622 [00:00<?, ? examples/s]

Adding EOS to train dataset:   0%|          | 0/1622 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/1622 [00:00<?, ? examples/s]

Truncating train dataset:   0%|          | 0/1622 [00:00<?, ? examples/s]

...

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: dhanya to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


/usr/local/lib/python3.11/dist-packages/trl/trainer/utils.py:177: UserWarning: Could not find response key `{output}` in the following instance: <s> ### Instruction:
Write a Taylor Swift-style lyric:
### Response:
We learn to live with the pain
Mosaic broken hearts
But this love is brave and wild</s></s></s></s></s></s></s></s></s></s></s></s></s></s></s>. This instance will be ignored in loss calculation. Note, if this happens often, consider increasing the `max_length`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/trl/trainer/utils.py:192: UserWarning: Could not find instruction key `### Instruction:
{instruction}
### Response:
` in the following instance: <s> ### Instruction:
Write a Taylor Swift-style lyric:
### Response:
We learn to live with the pain
Mosaic broken hearts
But this love is brave and wild</s></s></s></s></s></s></s></s></s></s></s></s></s></s></s>. This instance will be ignored in loss calculation. Note, if this happens often, consider increasing the `ma

Step,Training Loss
10,0.000000
20,0.000000
30,0.000000
40,0.000000
50,0.000000
60,0.000000
70,0.000000
80,0.000000
90,0.000000
100,0.000000


Streaming output truncated to the last 5000 lines.
Write a Taylor Swift-style lyric:
### Response:
I'll be usin' for the rest of my life
I've been breakin' hearts a long time
And toyin' with them older guys</s></s></s></s></s></s></s></s>. This instance will be ignored in loss calculation. Note, if this happens often, consider increasing the `max_length`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/trl/trainer/utils.py:192: UserWarning: Could not find instruction key `### Instruction:
{instruction}
### Response:
` in the following instance: <s> ### Instruction:
Write a Taylor Swift-style lyric:
### Response:
I'll be usin' for the rest of my life
I've been breakin' hearts a long time
And toyin' with them older guys</s></s></s></s></s></s></s></s>. This instance will be ignored in loss calculation. Note, if this happens often, consider increasing the `max_length`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/trl/trainer/utils.py:177: UserWarning: Could not find r

TrainOutput(global_step=1218, training_loss=0.0, metrics={'train_runtime': 512.7023, 'train_samples_per_second': 9.491, 'train_steps_per_second': 2.376, 'total_flos': 1882120215379968.0, 'train_loss': 0.0})

In [ ]:
prompt = "Write a Taylor Swift-style lyric:"
input_text = f"### Instruction:\n{prompt}\n### Response:\n"

inputs = tokenizer(input_text, return_tensors="pt").to("cuda")
outputs = model.generate(**inputs, max_new_tokens=50, do_sample=True, temperature=0.7)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

### Instruction:
Write a Taylor Swift-style lyric:
### Response:
I’ll be a little late to the party
But you won’t know how much I regret
Lost my rhythm, my rhyme
But I’m not a singer, I’m a song

I didn’
